In [1]:
%matplotlib inline

In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd

plt.ion()   # interactive mode

In [2]:
import pickle

In [3]:
train_imgs = pickle.load(open("train_images_512.pk",'rb'), encoding='bytes')
train_labels = pickle.load(open("train_labels_512.pk",'rb'), encoding='bytes')
test_imgs = pickle.load(open("test_images_512.pk",'rb'), encoding='bytes')


In [8]:
class CovidDatasetTrain(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, imgs, labels):
        self.imgs = imgs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.imgs[idx], self.labels[idx]

class CovidDatasetTest(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, imgs):
        self.imgs = imgs

    def __len__(self):
        return self.imgs.shape[0]

    def __getitem__(self, idx):
        return self.imgs[idx]  

In [9]:
def make_data_loaders():
    train_dataset = CovidDatasetTrain(train_imgs, train_labels)
    test_dataset = CovidDatasetTest(test_imgs)

    return {
        "train": DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1),
        "test": DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=1),
    }
    

In [10]:
data_loaders = make_data_loaders()
dataset_sizes = {'train': len(data_loaders['train'].dataset), 
                 'test':len(data_loaders['test'].dataset)}

class_names = ['covid', 'background']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
np.array(train_imgs).shape

X_messy = np.array(train_imgs)
X = np.zeros((70,512*512))
for i in range(70):
    for j in range(512):
        for k in range(512):
            X[i,j*512+k] = X_messy[i,1,j,k]
print(X.shape)
        
y = np.array(train_labels)

from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(max_depth=4, random_state=0)
#clf.fit(X, y)

(70, 262144)


In [28]:
X_messy = np.array(test_imgs)
Xtest = np.zeros((70,512*512))
for i in range(20):
    for j in range(512):
        for k in range(512):
            X[i,j*512+k] = X_messy[i,1,j,k]

In [30]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X, y)
yhat = clf.predict(X)
print(yhat)
print(y)
ytesthat = clf.predict

C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0
 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [11]:
#for sample in data_loaders["train"]:
#    print(sample[0].shape)
    
#things I know
#training: 15 covid, 55 not
#things I don't - how load data in their template stuff?
#print(train_imgs[35, :, 0,0])
training = np.array(train_imgs)
#print(train_imgs[:,:,:,:].var(axis = 1).sum())
print(len(train_labels[train_labels==0]))

15


In [4]:
toShuffle = np.concatenate((X,y[:,np.newaxis]), axis=1)
np.random.shuffle(toShuffle)
shuffled_y = toShuffle[:,-1] 
shuffled_X = toShuffle[:, :-1]

NameError: name 'X' is not defined

In [65]:
#cross validate?
a = np.arange(70)
#toShuffle = np.concatenate((X,y[:,np.newaxis]), axis=1)
#np.random.shuffle(toShuffle)
#shuffled_y = toShuffle[:,-1] 
#shuffled_X = toShuffle[:, 0:262143]
#print(shuffled_y)
#print(shuffled_X[:,-1])
#y
#np.random.shuffle(X) would mess with the labels...
tp = 0
fn = 0
fp = 0
clf = RandomForestClassifier(max_depth = 2, random_state=0)
for holdout in range(70):
    clf.fit(X[a%70 != holdout, :], y[a%70 != holdout])
    pred = clf.predict(X[np.newaxis, holdout, :])
    print(str(pred[0]) + ","+str(y[holdout]))
    if (pred[0] == 1 & y[holdout] == 1):
        tp += 1 
        print("tp")
    else:
        if((pred[0] == 1) & (y[holdout] == 0)):
            fp += 1
            print("fp")
        else:
            if((pred[0] == 0) & (y[holdout] == 1)):
                fn += 1
                print("fn")
                
print((fn+fp) / 70)

C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
0,0
0,0
1,0
fp
1,0
fp
0,0
1,0
fp
0,0
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
0,1
fn
1,0
fp
1,0
fp
0,0
0,0
0,0
1,0
fp
1,0
fp
1,0
fp
1,1
tp
1,1
tp
0,1
fn
1,1
tp
1,1
tp
0,1
fn
1,1
tp
0,1
fn
1,1
tp
0,1
fn
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
1,1
tp
0.18571428571428572


In [70]:
#toShuffle = np.concatenate((X,y[:,np.newaxis]), axis=1)
#np.random.shuffle(toShuffle)
#shuffled_y = toShuffle[:,-1] 
#shuffled_X = toShuffle[:, 0:262143]

a = np.arange(70)
for maxDepth in range(1, 10):
    tp = 0
    fn = 0
    fp = 0
    clf = RandomForestClassifier(max_depth = maxDepth, random_state=0)
    for holdout in range(70):
        #Xtraining = X[a%70 != holdout, :]
        #ytraining = y[a%70 != holdout, :]
        #Xvalidating = X[np.newaxis, holdout, :]
        clf.fit(X[a%70 != holdout, :], y[a%70 != holdout])
        pred = clf.predict(X[np.newaxis, holdout, :])
        #print(str(pred[0]) + ","+str(y[holdout]))
        if (pred[0] == 1 & y[holdout] == 1):
            tp += 1 
            #print("tp")
        else:
            if((pred[0] == 1) & (y[holdout] == 0)):
                fp += 1
                #print("fp")
            else:
                if((pred[0] == 0) & (y[holdout] == 1)):
                    fn += 1
                    #print("fn")
    p = tp/(tp+fn)
    r = tp/(tp+fp)
    f1 = 2*p*r/(p+r)
    print("MaxDepth: "+ str(maxDepth) + ", errorrate = "+ str((fp+fn)/70) + ", f1 = " + str(f1) + ", p = " + str(p) + ", r = " + str(r))
    
    
#best maxdepth is 3, with these hyperparameters

C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 1, errorrate = 0.2, f1 = 0.8793103448275862, p = 0.9272727272727272, r = 0.8360655737704918


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 2, errorrate = 0.18571428571428572, f1 = 0.8849557522123893, p = 0.9090909090909091, r = 0.8620689655172413


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

MaxDepth: 7, errorrate = 0.2, f1 = 0.8727272727272727, p = 0.8727272727272727, r = 0.8727272727272727


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 8, errorrate = 0.2, f1 = 0.8727272727272727, p = 0.8727272727272727, r = 0.8727272727272727


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 9, errorrate = 0.2, f1 = 0.8727272727272727, p = 0.8727272727272727, r = 0.8727272727272727


In [66]:
print(tp)
print(fn)
print(fp)
p = tp/(tp+fn)
print(p)
r = tp/(tp+fp)
print(r)
f1 = 2*p*r/(p+r)
print(f1)

50
5
8
0.9090909090909091
0.8620689655172413
0.8849557522123893


In [64]:
a = np.arange(70)
a
a%70 == 2

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [ ]:
#remember we need to evaluate cost

In [14]:
# Training loop starter
num_epochs = 1 # Set this yourself

for epoch in range(num_epochs):
    for sample in data_loaders["train"]:
    #    pass
    # Image shape
    # Batch size x Channels x Width x Height
        print(sample[0].shape)
    # Labels shape
    # Batch size
        print(sample[1].shape)

RuntimeError: DataLoader worker (pid(s) 15976) exited unexpectedly

In [20]:
import os
import pickle

from torchvision.utils import save_image

TRAIN_IMAGES_FILENAME = 'train_images_512.pk'
TRAIN_LABELS_FILENAME = 'train_labels_512.pk'
TEST_IMAGES_FILENAME = 'test_images_512.pk'


def load_data(adjust=True, save_images=False):
    """
    Load training and test data.
    Optionally re-scale data so it is between 0 and 1 (originally -1 and -0.9921).
    Optionally save images for visualization.
    :param adjust: Whether to re-scale data.
    :param save_images: Whether to save images.
    :return: train data, train labels, test data.
    """
    def load_pk(filename):
        with open(os.path.join(filename), 'rb') as f:
            return pickle.load(f, encoding='bytes')

    def adjust_window(imgs, shift=1, scale=127.5):
        return (imgs + shift) * scale

    train_imgs_ = load_pk(TRAIN_IMAGES_FILENAME)
    train_labels_ = load_pk(TRAIN_LABELS_FILENAME)
    test_imgs_ = load_pk(TEST_IMAGES_FILENAME)

    if adjust:
        train_imgs_ = adjust_window(train_imgs_)
        test_imgs_ = adjust_window(test_imgs_)

    if save_images:
        if not os.path.exists(os.path.join('..', 'images')):
            os.makedirs(os.path.join('..', 'images'))
        #for i, (img, label) in enumerate(zip(train_imgs_, train_labels_)):
        #    filename = os.path.join('..', 'images', 'yo.png')  #f'train_{i:02d}_{int(label):d}.png')
        #    if not os.path.exists(filename):
        #        save_image(img, filename)
        for i, img in enumerate(test_imgs_):
            filename = os.path.join('..', 'images', f'test_{i:02d}_{"":s}.png')
            if not os.path.exists(filename):
                save_image(img, filename)

    return train_imgs_, train_labels_, test_imgs_


if __name__ == '__main__':
    train_imgs, train_labels, test_imgs = load_data()

In [21]:
load_data(save_images=True)

(tensor([[[[1.0000, 0.9961, 0.8667,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.9961, 0.8771,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.9961, 0.8850,  ..., 0.2797, 0.2601, 0.2601],
           ...,
           [1.0000, 0.9882, 0.6784,  ..., 0.4078, 0.4039, 0.4000],
           [0.9961, 0.9882, 0.6810,  ..., 0.4000, 0.4039, 0.4078],
           [1.0000, 0.9882, 0.6784,  ..., 0.4078, 0.4118, 0.4196]],
 
          [[1.0000, 0.9961, 0.8667,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.9961, 0.8771,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.9961, 0.8850,  ..., 0.2797, 0.2601, 0.2601],
           ...,
           [1.0000, 0.9882, 0.6784,  ..., 0.4078, 0.4039, 0.4000],
           [0.9961, 0.9882, 0.6810,  ..., 0.4000, 0.4039, 0.4078],
           [1.0000, 0.9882, 0.6784,  ..., 0.4078, 0.4118, 0.4196]],
 
          [[1.0000, 0.9961, 0.8667,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.9961, 0.8771,  ..., 0.2758, 0.2601, 0.2601],
           [1.0000, 0.99